In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
gpu_available = tf.config.list_physical_devices('GPU')
print(gpu_available)
print(tf.__version__)
# tf.debugging.set_log_device_placement(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.16.2


In [3]:
# Initialising CNN
classifier = Sequential()

In [4]:
classifier.add(keras.Input(shape=(64, 64, 3)))
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# Step 2 - Pooling
classifier.add(MaxPool2D(pool_size=(2, 2)))
# Add one more Convolution layer
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Fully connected layer
classifier.add(Dense(128, activation='relu'))
# Output layer
classifier.add(Dense(1, activation='sigmoid'))
# Compile the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # metrics=[
    #     keras.metrics.BinaryAccuracy(),
    #     keras.metrics.FalseNegatives(),
    # ],
# Print the model summary
classifier.summary()

2024-08-15 08:55:43.612070: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-08-15 08:55:43.612089: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-08-15 08:55:43.612095: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-08-15 08:55:43.612113: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-15 08:55:43.612128: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 813,217 (3.10 MB)

 Trainable params: 813,217 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Part 2 - Fitting the CNN to the images
batch_size = 32

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/Users/adwiz/Downloads/Part 2 - Convolutional Neural Networks/dataset/training_set',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/Users/adwiz/Downloads/Part 2 - Convolutional Neural Networks/dataset/test_set',
        target_size=(64, 64),
        batch_size=batch_size,
        class_mode='binary')

classifier.fit(
        train_generator,
        steps_per_epoch=8000 // batch_size,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=2000 // batch_size)

classifier.save_weights('models/first_try.weights.h5')  # always save your weights after training or during training

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25


2024-08-15 08:55:44.093793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.5528 - loss: 0.6988 - val_accuracy: 0.6452 - val_loss: 0.6464
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5625 - val_loss: 0.6351
Epoch 3/25


2024-08-15 08:55:52.553041: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-15 08:55:52.553058: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/add_12/_44]]
2024-08-15 08:55:52.553074: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7003988479217762494
2024-08-15 08:55:52.553086: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10917822525070475908
2024-08-15 08:55:52.553090: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8812276392525590537
2024-08-15 08:55:52.553101: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 107216475

250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.6434 - loss: 0.6327 - val_accuracy: 0.6653 - val_loss: 0.6144
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 58us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.3923
Epoch 5/25


...

250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - accuracy: 0.7928 - loss: 0.4384 - val_accuracy: 0.7651 - val_loss: 0.5008
Epoch 22/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 53us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8125 - val_loss: 0.3254
Epoch 23/25


2024-08-15 08:57:10.191476: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-15 08:57:10.191493: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10917822525070475908
2024-08-15 08:57:10.191497: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/ReadVariableOp_4/_19]]
2024-08-15 08:57:10.191507: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8812276392525590537
2024-08-15 08:57:10.191511: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11688988086651523651
2024-08-15 08:57:10.191515: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash

250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.8041 - loss: 0.4230 - val_accuracy: 0.7959 - val_loss: 0.4482
Epoch 24/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 53us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 0.5491
Epoch 25/25


2024-08-15 08:57:17.779308: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-15 08:57:17.779329: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/gradient_tape/compile_loss/binary_crossentropy/mod/_51]]
2024-08-15 08:57:17.779344: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7003988479217762494
2024-08-15 08:57:17.779352: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8812276392525590537
2024-08-15 08:57:17.779355: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 341756783680733788
2024-08-15 08:57:17.779360: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous rec

250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.8027 - loss: 0.4158 - val_accuracy: 0.8014 - val_loss: 0.4400
